# Future Strategy - Signal Event 0002 - Indicator Timing Strategy

## 1. Import Library

In [8]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [9]:
sd = '2023-01-01'
ed = '2026-02-13'

In [10]:
instrument_list = ['AP8888.CZC']

In [11]:
sql_strategy =  f"""
SELECT
    date,
    instrument,
    m_ta_ema(close, 5) AS ma_short,
    m_ta_ema(close, 20) AS ma_long,
    IF(ma_short > ma_long AND m_lag(ma_short, 1) < m_lag(ma_long, 1), 1, 0) AS signal_long,
    IF(ma_short < ma_long AND m_lag(ma_short, 1) > m_lag(ma_long, 1), 1, 0) AS signal_short,
FROM cn_future_bar1d
"""

In [12]:
sql_trade = f"""
WITH 
data_signal AS (
    {sql_strategy}
)
SELECT
    date,
    instrument,
    signal_long,
    signal_short,
FROM data_signal
WHERE (signal_long = 1 OR signal_short = 1)
AND instrument IN {str(tuple(instrument_list))}
ORDER BY date, instrument
"""

In [13]:
df = dai.query(sql_trade, filters={"date":[sd, ed]}).df()
df

,date,instrument,signal_long,signal_short
0,2023-03-14,AP8888.CZC,0,1
1,2023-04-20,AP8888.CZC,1,0
2,2023-05-16,AP8888.CZC,0,1
3,2023-05-18,AP8888.CZC,1,0
4,2023-05-24,AP8888.CZC,0,1
5,2023-05-25,AP8888.CZC,1,0
6,2023-06-07,AP8888.CZC,0,1
7,2023-06-08,AP8888.CZC,1,0
8,2023-06-12,AP8888.CZC,0,1
9,2023-06-29,AP8888.CZC,1,0


## 3. Backtest Strategy

In [14]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 3

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]
    
    from bigtrader.constant import OrderType
    from bigtrader.constant import Direction

    instruments_hold  = set(context.get_account_positions().keys())

    for ins in instruments_hold:
        position_long  = context.get_position(ins, Direction.LONG)
        position_short = context.get_position(ins, Direction.SHORT)
        price = data.current(ins,"open")
        if (position_long.current_qty  != 0):
            context.sell_close(ins, position_long.avail_qty, price, order_type=OrderType.MARKET)
        if (position_short.current_qty != 0):
            context.buy_close(ins, position_short.avail_qty, price, order_type=OrderType.MARKET)

    if len(df_now) == 0:
        return

    instruments_long  = set(df_now[df_now["signal_long"]  == 1]["instrument"])
    instruments_short = set(df_now[df_now["signal_short"] == 1]["instrument"])

    for ins in instruments_long:
        price = data.current(ins, "open")
        volume = 1
        context.buy_open(ins, volume, price, order_type=OrderType.MARKET)
    
    for ins in instruments_short:
        price = data.current(ins, "open")
        volume = 1
        context.sell_open(ins, volume, price, order_type=OrderType.MARKET)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 10000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""期货""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-26 21:16:59] [info     ] bigtrader.v34 开始运行 ..
[2026-02-26 21:16:59] [info     ] 2023-03-14, 2026-02-04, , future, instruments=1
[2026-02-26 21:16:59] [info     ] bigtrader module V2.2.0
[2026-02-26 21:16:59] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-26 21:16:59] [info     ] backtest done, raw_perf_ds:dai.DataSource("_a1304d5d738145f0b7a432a12ea10353")


[2026-02-26 21:17:00] [info     ] bigtrader.v34 运行完成 [0.958s].
